# check oof df

In [1]:
import os
import gc
import yaml
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("/kaggle/src/dss_utils")
sys.path.append("/kaggle/src/exp")
sys.path.append("/kaggle/src/model")
sys.path.append("/kaggle/src/data")
from dss_metrics import score
from training_loop import get_valid_values_dict, concat_valid_input_info, get_oof_df, get_key_df
from dss_model import get_model
from dss_dataloader import get_loader


In [2]:
output_dir = "/kaggle/working"
# exp_name = "exp022_dense_skffold_removesomeseries_epoch10"
exp_name = "exp023_alldata_skf_dense_epoch30"
folds = [0, 1, 2, 3, 4]
# folds = [0]
oof_df = pd.DataFrame()
for i in folds:
    print("fold", i)
    # df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"raw_oof_df_fold{i}.parquet"))
    df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"fold{i}_best_oof_df.parquet"))
    oof_df = pd.concat([oof_df, df], axis=0)


print(len(oof_df["series_id"].unique()))   


fold 0
fold 1
fold 2
fold 3
fold 4
277


In [3]:
oof_df = oof_df.sort_values(["series_id", "step"])

oof_df = oof_df[["series_id", "step", "second", "event_pred", "class_pred", "class_det_pred"]]

display(oof_df)


,series_id,step,second,event_pred,class_pred,class_det_pred
0,038441c925bb,0.0,0,0.202160,0.260857,0.607000
1,038441c925bb,1.0,5,0.000000,0.149328,0.648476
2,038441c925bb,2.0,10,0.000000,0.125631,0.665744
3,038441c925bb,3.0,15,0.000000,0.134012,0.662744
4,038441c925bb,4.0,20,0.000000,0.147043,0.662563
...,...,...,...,...,...,...
25500432,fe90110788d2,592375.0,35,0.000000,0.048466,0.378186
25500433,fe90110788d2,592376.0,40,0.000000,0.047598,0.378881
25500434,fe90110788d2,592377.0,45,0.000000,0.046730,0.380233
25500435,fe90110788d2,592378.0,50,0.000000,0.046055,0.382480


In [22]:
# postprocess_fn
# series_idでgroupbyして、class_predに対して対象の列のデータから前のN個の列までのデータの平均をとる
import torch
import torch.nn as nn

def detect_event_from_classpred(df, N=300, maxpool_kernel_size=41, det_threshold=0.4):
    df = df.copy()

    # series_idでgroupbyして、class_predに対して対象の列のデータから
    # 前のN個の列までのデータの平均をとる
    df["class_pred_beforemean"] = df.groupby("series_id")["class_pred"].apply(
        lambda x: x.rolling(N, min_periods=1).mean()
    )
    df["class_pred_aftermean"] = df.groupby("series_id")["class_pred"].apply(
        lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1]
    )
    df["event_pred"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]

    # 入力サイズと出力サイズが一致するようにpaddingを調整
    maxpool_padding = int((maxpool_kernel_size - 1) / 2)
    # maxpoolしてピーク検出
    max_pooling = nn.MaxPool1d(
        maxpool_kernel_size, stride=1, padding=maxpool_padding
    )
    event_pred = df["event_pred"].values
    event_pred = torch.tensor(event_pred).unsqueeze(0)
    pooled_event_pred = max_pooling(np.abs(event_pred)).squeeze(0).numpy()
    event_pred = event_pred.squeeze(0).numpy()
    # peakのところだけ残すmaskを作成
    peak_event_pred_mask = np.where(pooled_event_pred == np.abs(event_pred), 1, 0)
    peak_event_pred = event_pred * peak_event_pred_mask

    # class_det_pred = df["class_det_pred"].values
    # class_det_pred = np.where(class_det_pred > det_threshold, np.ones_like(class_det_pred), class_det_pred)
    # class_det_pred = np.where(class_det_pred > det_threshold, np.ones_like(class_det_pred), np.zeros_like(class_det_pred))
    # class det predをrolling mean
    df["class_det_pred"] = df.groupby("series_id")["class_det_pred"].apply(
        lambda x: x.rolling(60, min_periods=1).mean()
    )
    class_det_pred = df["class_det_pred"].values
    df["event_pred"] = peak_event_pred * class_det_pred
    df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)

    return df

def postprocess_notrolling(df):
    df = df.copy()
    # 一つ前のclass_predを取得
    df["class_pred_before"] = df.groupby("series_id")["class_pred"].shift(1)
    # 一つ後のclass_predを取得
    df["class_pred_after"] = df.groupby("series_id")["class_pred"].shift(-1)
    df["event_pred"] = df["class_pred_before"] - df["class_pred_after"]
    not_predicted_mask = (df["class_pred"] != -1).astype(int)
    df["event_pred"] = df["event_pred"] * not_predicted_mask
    df = df.drop(["class_pred_before", "class_pred_after"], axis=1)
    return df

def make_submission_df(df, threshold=0.1):
    df = df[["series_id", "step", "event_pred"]].copy()
    # thresholdより大きいときは1,-thresholdより小さいときは-1,それ以外は0
    df["event"] = df["event_pred"].apply(lambda x: 1 if x > threshold else -1 if x < -threshold else 0)
    df = df[df["event"] != 0].copy()
    df["event"] = df["event"].replace({1: "wakeup", -1: "onset"})
    df["score"] = df["event_pred"].apply(lambda x: np.clip(np.abs(x), 0.0, 1.0))
    return df


In [5]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
train_event_df = train_event_df[train_event_df["step"].notnull()].copy()
# train_event_df = train_event_df[train_event_df["series_id"].isin(oof_df["series_id"].unique())].copy()
train_event_df = train_event_df.reset_index(drop=True)
display(train_event_df.head())


,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [15]:
%%time
max_kernel_size = 3
tmp_score = 0
average_size_ = 61
max_kernel_size = 3
print("average_size", average_size_, "max_kernel_size", max_kernel_size)
for det_thr in range(1, 10, 1):
    print("det_thr", det_thr/10)
    df = detect_event_from_classpred(oof_df, N=average_size_, maxpool_kernel_size=max_kernel_size, det_threshold=det_thr/10)
    sub_df = make_submission_df(df, threshold=0.1)
    if len(sub_df) == 0:
        print("event not detected")
        continue
    print(len(sub_df))
    score_ = score(train_event_df, sub_df)
    print(score_)
    if tmp_score > score_:
        break
    tmp_score = score_
    average_size = average_size_


average_size 61 max_kernel_size 3
det_thr 0.1
27386
0.6954145404020978
det_thr 0.2
27017
0.6951206303002108
CPU times: user 1min 53s, sys: 17.2 s, total: 2min 10s
Wall time: 2min 7s


In [10]:
# %%time
# tmp_score = 0
# average_size_ = 132 
# max_kernel_size = 43
# print("average_size", average_size_, "max_kernel_size", max_kernel_size)
# for det_thr in range(1, 10, 1):
#     print("det_thr", det_thr/10)
#     df = detect_event_from_classpred(oof_df, N=average_size_, maxpool_kernel_size=max_kernel_size, det_threshold=det_thr/10)
#     sub_df = make_submission_df(df, threshold=0.1)
#     if len(sub_df) == 0:
#         print("event not detected")
#         continue
#     print(len(sub_df))
#     score_ = score(train_event_df, sub_df)
#     print(score_)
#     if tmp_score > score_:
#         break
#     tmp_score = score_
#     average_size = average_size_


average_size 132 max_kernel_size 43
det_thr 0.1
class_det_pred (127950909,)
21012
0.7134946264945694
det_thr 0.2
class_det_pred (127950909,)
20788
0.7128487693797367
CPU times: user 1min 59s, sys: 17.5 s, total: 2min 16s
Wall time: 2min 13s


In [23]:
average_size = 132 
max_kernel_size = 43
df = detect_event_from_classpred(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size, det_threshold=0.1)
sub_df = make_submission_df(df, threshold=0.01)
print(average_size, max_kernel_size)

score_ = score(train_event_df, sub_df)
print(score_)


132 43
0.7201965751684345


ones or values  0.7217737664056836

ones or zeros  0.7217709266197103

values 0.7180762860353304